In [1]:
import pandas as pd
import simpy
import numpy as np
import random

In [37]:
class Balancer(object):
    def __init__(self, env, count, flag_RR):
        self.env = env
        self.count = count
        self.servers = [simpy.Resource(env) for x in range(count)]
        self.flag_RR = flag_RR
        self.nextServer = 0

    def serve(self, client):
        if self.flag_RR:
            server = self.select_server_RR()
        else:
            server = self.select_server_not_bussy() 
        global queue
        if (queue < len(server.queue)):
            queue = len(server.queue)
        with server.request() as req:
            yield req
            yield self.env.timeout(client.get_service_time())

    def select_server_RR(self):
        current = self.nextServer
        self.nextServer += 1
        if (current == 4):
            self.nextServer = 0

        return self.servers[current]
    
    def select_server_not_bussy(self):
        min_server = self.servers[0]
        for server in self.servers:
            if (len(server.queue) < len(min_server.queue)):
                min_server = server 

        return min_server


class Client(object):

    def __init__(self, env):
        self.env = env
        self.type = np.random.choice(['A', 'B', 'C'], p=[0.7, 0.2, 0.1])
        self.service_time =  self.service_time = {
            'A': random.uniform(60,180),  
            'B': random.uniform(120,360),
            'C': random.uniform(200,800)
        }.get(self.type)
        self.arrive = 0
        
    def release(self, balancer):
        self.arrive = self.env.now
        yield self.env.process(balancer.serve(self))
        wait = self.env.now - self.arrive
        global max_wait_time
        if (max_wait_time < wait):
            max_wait_time = wait
        print(f"%.2f Type {self.type}  attended" % (self.env.now))
        
    def get_service_time(self):
        return self.service_time


servers = 5
arrival_rate = 0.045
client_count = 100
queue = 0
max_wait_time = 0

def generate_clients(environment, count, interval, checkout):
    for i in range(count):
        client = Client(env)
        environment.process(client.release(checkout))
        yield environment.timeout(np.random.exponential(interval))


87.41 Type A  attended
137.49 Type A  attended
165.98 Type A  attended
175.88 Type A  attended
244.55 Type A  attended
314.98 Type A  attended
330.11 Type A  attended
353.01 Type B  attended
417.98 Type A  attended
432.10 Type B  attended
434.65 Type A  attended
492.78 Type A  attended
517.33 Type A  attended
558.23 Type A  attended
575.72 Type C  attended
580.83 Type A  attended
610.73 Type A  attended
663.03 Type A  attended
683.03 Type B  attended
684.84 Type A  attended
715.53 Type A  attended
831.31 Type A  attended
910.81 Type A  attended
935.66 Type B  attended
939.62 Type B  attended
960.53 Type C  attended
1006.21 Type A  attended
1074.00 Type B  attended
1120.97 Type A  attended
1124.49 Type A  attended
1197.78 Type B  attended
1216.87 Type B  attended
1253.05 Type C  attended
1253.06 Type A  attended
1305.63 Type A  attended
1380.30 Type C  attended
1401.18 Type A  attended
1406.78 Type B  attended
1411.25 Type A  attended
1470.63 Type A  attended
1481.35 Type A  attended
15

### Round Robin version

In [40]:
env = simpy.Environment()
balancer = Balancer(env, servers, True)
env.process(generate_clients(env, client_count, arrival_rate, balancer))
env.run()

print(f"Max queue: {queue}" )
print(f"wait time maximun: {max_wait_time}")

96.60 Type A  attended
133.52 Type A  attended
156.89 Type B  attended
180.34 Type B  attended
242.39 Type A  attended
248.70 Type A  attended
283.68 Type A  attended
326.00 Type A  attended
330.07 Type B  attended
331.23 Type C  attended
440.18 Type A  attended
494.97 Type B  attended
501.94 Type A  attended
539.53 Type A  attended
543.33 Type B  attended
575.85 Type B  attended
632.18 Type B  attended
645.96 Type A  attended
669.45 Type A  attended
701.33 Type A  attended
706.10 Type A  attended
779.01 Type A  attended
785.94 Type B  attended
789.28 Type A  attended
847.38 Type A  attended
851.76 Type A  attended
868.22 Type B  attended
913.79 Type A  attended
923.54 Type A  attended
957.81 Type A  attended
982.94 Type A  attended
1000.25 Type A  attended
1084.77 Type A  attended
1095.58 Type B  attended
1147.97 Type A  attended
1163.24 Type B  attended
1163.91 Type A  attended
1231.67 Type A  attended
1252.52 Type A  attended
1299.84 Type A  attended
1301.13 Type A  attended
1341.12

### Least busy server

In [38]:
env = simpy.Environment()
balancer = Balancer(env, servers, False)
env.process(generate_clients(env, client_count, arrival_rate, balancer))
env.run()

print(f"Max queue: {queue}" )
print(f"wait time maximun: {max_wait_time}")

60.76 Type A  attended
137.75 Type A  attended
139.00 Type A  attended
148.45 Type A  attended
152.06 Type A  attended
249.96 Type A  attended
271.96 Type A  attended
318.99 Type A  attended
376.94 Type A  attended
386.82 Type A  attended
408.03 Type A  attended
506.83 Type C  attended
545.47 Type A  attended
565.92 Type A  attended
570.70 Type C  attended
630.38 Type A  attended
631.37 Type A  attended
663.81 Type A  attended
672.21 Type C  attended
693.26 Type A  attended
711.20 Type A  attended
734.18 Type A  attended
738.16 Type A  attended
782.19 Type A  attended
800.74 Type B  attended
837.05 Type A  attended
892.34 Type A  attended
896.54 Type A  attended
923.65 Type A  attended
972.51 Type A  attended
1009.85 Type A  attended
1103.09 Type B  attended
1112.22 Type A  attended
1187.52 Type A  attended
1198.43 Type A  attended
1353.80 Type A  attended
1401.18 Type B  attended
1467.29 Type A  attended
1479.90 Type C  attended
1491.72 Type A  attended
1509.65 Type C  attended
1523.3